In [21]:
!pip install tqdm


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import torch
import torchaudio

import pandas as pd
import numpy as np

import os

from tqdm.auto import tqdm

import IPython

from torch import nn
import torch.nn.functional as F

from modules.dataset import AudioDataset

from torch.utils.data import DataLoader


from transformers import PreTrainedTokenizerFast

from modules.model import Transformer
from modules.tokenizer import tokenize

In [2]:
path_to_tokenizer = './tokenizer.json'
path_to_data = './audio_dataset/'

In [3]:
data = pd.read_csv(os.path.join(path_to_data,'df.csv'), usecols=['text','status','path','rate','duration','frames'])
data = data[data.status=='APPROVED'].reset_index(drop=True)
del data['status']
data.text = data.text.apply(lambda x: "".join([char for char in x if char.isalpha() or char==' ']).lower())

In [4]:
data.duration.max()

22.06

In [5]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.displot(data.text.str.len())
# plt.show()dd
# sns.displot(data.duration)
# plt.show()

In [6]:
np.percentile(data.text.str.len(), 99.5)

135.0

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = PreTrainedTokenizerFast(tokenizer_file=path_to_tokenizer, padding_side ='right', pad_token ='[PAD]')
tokenizer.bos_token = '[SOS]'
tokenizer.eos_token = '[EOS]'
tokenizer.pad_token = '[PAD]'
tokenizer.unk_token = '[UNK]'
tokenizer.mask_token = "[MASK]"
dataset = AudioDataset(data, path_to_data, tokenizer, n_fft=1024, n_mels=64, center=True, max_tokenized_length=100, max_audio_len=25, sr=16000)
model = Transformer(vocab_size=len(tokenizer),
                    n_mels=64,
                    enc_seq_len=25, 
                    dec_seq_len=100,
                    hidden_dim=32, 
                    enc_num_layers=2, 
                    dec_num_layers=2, 
                    num_heads=3, 
                    ff_dim=128, 
                    r_dim=100, 
                    dropout=0.1, 
                    sr=16000, 
                    n_fft=1024)

n_fft=1024, win_lenght=1024, hop_lenght=256, n_mels=64, center=True):

In [25]:
dataloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0)
for i_batch, sample_batched in tqdm(enumerate(dataloader)):
    padded_input = model.predict(sample_batched, tokenizer.bos_token_id, tokenizer.eos_token_id)
    print(tokenizer.decode(padded_input.squeeze()))
    break

0it [00:00, ?it/s]

[SOS] [EOS] [SOS] [EOS] [PAD] [PAD] [PAD] [EOS] [PAD] [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [SOS] [PAD] [PAD] [EOS] [EOS] [PAD] [EOS] [PAD] [EOS] [SOS] [EOS] [PAD] ñ [SOS] [EOS] [PAD] [EOS] w [SOS] w w ß ã [EOS] ä è ê w è ñ [EOS] ä ã [EOS] è ñ [SOS] [SOS] ä ä è è ä ã ò w w ő ß ã ß ß [PAD] ő w ж ж ê [EOS] ß ä ő [PAD] w è [SOS] ж ã ä [EOS] ê ê [PAD] ß ő ß ã w é w [PAD] ư ж э ò
